In [8]:
import ligpy_utils as lig
import constants as const

In [9]:
T = 300
species_IC = 'Pseudotsuga_menziesii'

In [10]:
reactionlist, rateconstantlist, compositionlist = lig.set_paths()
y_list = lig.get_specieslist(reactionlist)

In [11]:
speciesindices, indices_to_species = lig.get_speciesindices(y_list)
rate_list = lig.build_rates_list(rateconstantlist, reactionlist, speciesindices, indices_to_species, T)
species_rxns = lig.build_species_rxns_dict(reactionlist)
dydt_expressions = lig.build_dydt_list(rate_list, y_list, species_rxns)
kmatrix = lig.build_k_matrix(rateconstantlist)
kvaluelist = lig.get_k_value_list(T, kmatrix)
PLIGC_0, PLIGH_0, PLIGO_0 = lig.define_initial_composition(compositionlist, species_IC)

In [22]:
with open('solve_ODEs_T%s.py' %T, 'w') as f:
    text = "#!/usr/bin/pythons\n" \
            "# -*- coding: utf-8 -*-\n\n" \
            "from scipy.integrate import odeint\n\n\n" \
            "def ODEs(y, t, p):\n"
    f.write(text)
    
    y = '\t' + y_list[0]
    for i in y_list[1:]:
        y += (', ' + i)
    y += ' = y'
    f.write(y + '\n')
    
    p = '\tk_%s_0' %T
    for i in range(1, len(kvaluelist)):
        p += ', k_%s_%s' %(T, i)
    p += ' = p'
    f.write(p + '\n')
    
    dydt = '\tdydt = [%s' %dydt_expressions[0].split('=')[1][1:-2]
    for ODE in dydt_expressions[1:]:
        dydt += (', \n\t\t\t' + ODE.split('=')[1][1:-2])
    dydt += ']'
    f.write(dydt + '\n')
    
    f.write('\treturn dydt\n\n')
    f.write('# k values\n')
    for i in range(len(kvaluelist)):
        f.write('k_%s_%s = %s\n' %(T, i, kvaluelist[i]))
        
    f.write('\n# Initial conditions\n')
    f.write('PLIGC = %s\nPLIGH = %s\nPLIGO = %s\n' %(PLIGC_0, PLIGH_0, PLIGO_0))
    IC = ''
    for i in y_list:
        if i == 'PLIGC' or i == 'PLIGH' or i == 'PLIGO':
            continue
        else:
            IC += ('%s = ' %i)
    IC += '0'
    f.write(IC + '\n\n')
    
    f.write('# ODE solver parameters\n')
    f.write('abserr = %s\n' %const.ABSOLUTE_TOLERANCE)
    f.write('relerr = %s\n' %const.RELATIVE_TOLERANCE)
    f.write('stoptime = 2000\n')
    f.write('numpoints = 10000\n\n')
    
    f.write('t = [stoptime * float(i) / (numpoints - 1) for i in range(numpoints)]\n')
    y0 = 'y0 = [' + y_list[0]
    for spec in y_list[1:]:
        y0 += ', %s' %spec
    y0 += ']'
    f.write(y0)
    
    p = '\np = [k_%s_0' %T
    for i in range(1, len(kvaluelist)):
        p += ', k_%s_%s' %(T, i)
    p += ']\n'
    f.write(p)
    
    f.write('\nsol = odeint(ODEs, y0, t, args=(p,), atol=abserr, rtol=relerr)\n\n')
    
    f.write("with open('sol_T%s_%s.dat', 'w') as f:\n" %(T, species_IC))
    
    ysol = ''
    for i in range(len(y_list)):
        ysol += 'yy[%s], ' %i
    f.write('\tfor tt, yy in zip(t, ysol):\n')
    f.write('\t\tprint(tt, %sfile=f)\n' %ysol)

In [6]:
y0

'y0 = [ADIO, ADIOM2, ALD3, C10H2, C10H2M2, C10H2M4, C2H6, C3H4O, C3H4O2, C3H6, C3H6O2, C3H8O2, CH2CO, CH3CHO, CH3OH, CH4, CHAR, CO, CO2, COUMARYL, ETOH, H2, H2O, KET, KETD, KETDM2, KETM2, LIG, LIGC, LIGH, LIGM2, LIGO, MGUAI, OH, PADIO, PADIOM2, PC2H2, PCH2OH, PCH2P, PCH3, PCHO, PCHOHP, PCHP2, PCOH, PCOHP2, PCOS, PFET3, PFET3M2, PH2, PHENOL, PKETM2, PLIG, PLIGC, PLIGH, PLIGM2, PLIGO, PRADIO, PRADIOM2, PRFET3, PRFET3M2, PRKETM2, PRLIGH, PRLIGH2, PRLIGM2A, RADIO, RADIOM2, RC3H3O, RC3H5O2, RC3H7O2, RCH3, RCH3O, RKET, RKETM2, RLIGA, RLIGB, RLIGH, RLIGM2A, RLIGM2B, RMGUAI, RPHENOL, RPHENOX, RPHENOXM2, SYNAPYL, VADIO, VADIOM2, VCOUMARYL, VKET, VKETD, VKETDM2, VKETM2, VMGUAI, VPHENOL, VSYNAPYL]'

In [18]:
ysol

'yy[0], yy[1], yy[2], yy[3], yy[4], yy[5], yy[6], yy[7], yy[8], yy[9], yy[10], yy[11], yy[12], yy[13], yy[14], yy[15], yy[16], yy[17], yy[18], yy[19], yy[20], yy[21], yy[22], yy[23], yy[24], yy[25], yy[26], yy[27], yy[28], yy[29], yy[30], yy[31], yy[32], yy[33], yy[34], yy[35], yy[36], yy[37], yy[38], yy[39], yy[40], yy[41], yy[42], yy[43], yy[44], yy[45], yy[46], yy[47], yy[48], yy[49], yy[50], yy[51], yy[52], yy[53], yy[54], yy[55], yy[56], yy[57], yy[58], yy[59], yy[60], yy[61], yy[62], yy[63], yy[64], yy[65], yy[66], yy[67], yy[68], yy[69], yy[70], yy[71], yy[72], yy[73], yy[74], yy[75], yy[76], yy[77], yy[78], yy[79], yy[80], yy[81], yy[82], yy[83], yy[84], yy[85], yy[86], yy[87], yy[88], yy[89], yy[90], yy[91], yy[92], '

In [20]:
p

'\tk_300_0, k_300_1, k_300_2, k_300_3, k_300_4, k_300_5, k_300_6, k_300_7, k_300_8, k_300_9, k_300_10, k_300_11, k_300_12, k_300_13, k_300_14, k_300_15, k_300_16, k_300_17, k_300_18, k_300_19, k_300_20, k_300_21, k_300_22, k_300_23, k_300_24, k_300_25, k_300_26, k_300_27, k_300_28, k_300_29, k_300_30, k_300_31, k_300_32, k_300_33, k_300_34, k_300_35, k_300_36, k_300_37, k_300_38, k_300_39, k_300_40, k_300_41, k_300_42, k_300_43, k_300_44, k_300_45, k_300_46, k_300_47, k_300_48, k_300_49, k_300_50, k_300_51, k_300_52, k_300_53, k_300_54, k_300_55, k_300_56, k_300_57, k_300_58, k_300_59, k_300_60, k_300_61, k_300_62, k_300_63, k_300_64, k_300_65, k_300_66, k_300_67, k_300_68, k_300_69, k_300_70, k_300_71, k_300_72, k_300_73, k_300_74, k_300_75, k_300_76, k_300_77, k_300_78, k_300_79, k_300_80, k_300_81, k_300_82, k_300_83, k_300_84, k_300_85, k_300_86, k_300_87, k_300_88, k_300_89, k_300_90, k_300_91, k_300_92, k_300_93, k_300_94, k_300_95, k_300_96, k_300_97, k_300_98, k_300_99, k_300_1

In [ ]:
b

In [ ]:
a = 3

In [ ]:
a

In [ ]:
b

In [ ]:
c